# ML-Project-2

## This project extends my chatbot demo by integrating it with the Endor AI platform for advanced dynamic learning and predictive modeling. The codebase is designed to be a reference implementation for deploying the chatbot on Endor, demonstrating how to preprocess text data, train a predictive model, implement adaptive conversation flows, and integrate everything into a Flask web application.

### Step 1 - Install libraries

In [3]:
pip install requests flask spacy nltk transitions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.7/96.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 45.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.0/183.0 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 634.4/634.4 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.2/774.2 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Step 2 - Data Collection and Preprocessing

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

def extract_features(text):
    doc = nlp(text)
    # Extract part-of-speech tags, named entities, etc.
    features = {
        "num_tokens": len(doc),
        "entities": [ent.label_ for ent in doc.ents],
        # More features can be added as needed
    }
    return features

sample_text = "I am happy with the service, but could be better."
print(extract_features(sample_text))

## ** Formatting Data for Endor **

### Combine structured and unstructured features into a JSON (or other required format) that matches Endor’s API specifications

### Step 3 - Training the Predictive Model with Endor

In [ ]:
import requests
import json

# Replace with your actual Endor API endpoint and key
ENDOR_API_URL = "https://api.endor.com/v1/train"
API_KEY = "your_api_key_here"

def train_model(training_data):
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }
    response = requests.post(ENDOR_API_URL, headers=headers, json=training_data)
    if response.status_code == 200:
        print("Model training initiated successfully!")
        return response.json()
    else:
        print("Error during training:", response.text)
        return None

# Example training data format (structure depends on Endor's API)
training_data = {
    "data": [
        # Your data entries here, combining structured fields and NLP-extracted features
    ],
    "target": "desired_outcome"
}

train_model(training_data)

### Step 4 - Building Adaptive Conversation Flows

### Implement State Management

In [ ]:
from transitions import Machine

class Chatbot:
    states = ['greeting', 'query_handling', 'escalation', 'farewell']

    def __init__(self):
        self.machine = Machine(model=self, states=Chatbot.states, initial='greeting')
        self.machine.add_transition(trigger='process_query', source='greeting', dest='query_handling')
        self.machine.add_transition(trigger='escalate', source='query_handling', dest='escalation')
        self.machine.add_transition(trigger='finish', source='*', dest='farewell')

    def current_state(self):
        return self.state

bot = Chatbot()
print("Initial state:", bot.current_state())
bot.process_query()
print("State after processing query:", bot.current_state())

### Step 5 - Dynamic Adaptation Based on Predictions

In [ ]:
def get_prediction(user_input):
    # Preprocess the input as needed (e.g., extract features)
    features = extract_features(user_input)
    # Prepare the payload for Endor API
    payload = {
        "features": features,
        # Other required fields per Endor’s API
    }
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }
    prediction_url = "https://api.endor.com/v1/predict"
    response = requests.post(prediction_url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()  # Assuming this returns the prediction outcome
    else:
        print("Error during prediction:", response.text)
        return None

# Example usage within a conversation flow
user_input = "I am not satisfied with my recent experience."
prediction = get_prediction(user_input)
if prediction:
    # Decide based on prediction result
    if prediction.get("sentiment") == "negative":
        bot.escalate()
        response_text = "I understand your concern. Let me connect you with our support team."
    else:
        bot.process_query()
        response_text = "Thanks for your feedback! How can I assist you further?"
else:
    response_text = "I'm having trouble understanding that. Could you please rephrase?"

print("Bot response:", response_text)

### Step 6 - Integrating Everything into a Flask Web App

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)
chatbot_instance = Chatbot()

@app.route("/chat", methods=["POST"])
def chat():
    user_message = request.json.get("message", "")
    # Get prediction from Endor
    prediction = get_prediction(user_message)
    
    # Determine response based on prediction
    if prediction and prediction.get("sentiment") == "negative":
        chatbot_instance.escalate()
        reply = "I’m sorry to hear that. Let me connect you with a support specialist."
    else:
        chatbot_instance.process_query()
        reply = "Thanks for sharing that. What else can I help you with?"
    
    return jsonify({"reply": reply, "state": chatbot_instance.current_state()})

if __name__ == "__main__":
    app.run(debug=True)